<a href="https://colab.research.google.com/github/duckbumbum301/biomed-assistant/blob/huydt/Llama3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/gemma-3-4b-it"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit=False,
    full_finetuning=False
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    finetune_vision_layers = False,
    finetune_language_layers = True,
    finetune_attention_modules = True,

    r = 8,
    lora_alpha=0.8,
    lora_dropout=0.1,
    bias = 'none',
    random_state = 3407
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [ ]:
import json
file_path = "/content/drive/MyDrive/data/MedicalChatbot/disease_data_message.json"

with open(file_path, "r", encoding="utf-8") as f:
    dataset = json.load(f)

print("Số phần tử:", len(dataset))
print("Item đầu:", dataset[0])

Số phần tử: 11
Item đầu: {'messages': [{'role': 'user', 'content': 'Mình muốn biết thông tin về bệnh Cảm lạnh thông thường. Hãy trích xuất các phần: tên bệnh, mô tả, triệu chứng và cách điều trị.'}, {'role': 'assistant', 'content': '{\n  "name": "Cảm lạnh thông thường",\n  "definition": "(phổ biến nhất là rhinovirus). Bệnh thường ảnh hưởng đến mũi và cổ họng. Cảm\\nlạnh thông thường không quá nguy hiểm nhưng có thể làm cho trẻ bị khó chịu và\\ngây gián đoạn một số hoạt động sinh hoạt hàng ngày. Đây là một trong những bệnh\\nthường gặp ở trẻ em. (1)",\n  "symptoms": "Triệu chứng của cảm lạnh thường bao gồm sổ mũi, nghẹt mũi, hắt hơi, ho, đau\\nhọng, sốt nhẹ, mệt mỏi và đôi khi đau nhức cơ thể nhẹ. Khi nhận thấy con có những\\ntriệu chứng này, ba mẹ hãy đảm bảo cho trẻ nghỉ ngơi nhiều, uống nhiều nước và\\ngiữ vệ sinh cá nhân tốt để ngăn ngừa lây lan virus sang người khác.",\n  "treatment": "Cách điều trị cảm lạnh hướng đến mục tiêu giảm triệu chứng và chăm sóc hỗ trợ để\\nnâng cao phản 

In [ ]:
# Convert list → Dataset
from datasets import Dataset

dataset = Dataset.from_list(dataset)

In [ ]:
dataset[0]

{'messages': [{'content': 'Mình muốn biết thông tin về bệnh Cảm lạnh thông thường. Hãy trích xuất các phần: tên bệnh, mô tả, triệu chứng và cách điều trị.',
   'role': 'user'},
  {'content': '{\n  "name": "Cảm lạnh thông thường",\n  "definition": "(phổ biến nhất là rhinovirus). Bệnh thường ảnh hưởng đến mũi và cổ họng. Cảm\\nlạnh thông thường không quá nguy hiểm nhưng có thể làm cho trẻ bị khó chịu và\\ngây gián đoạn một số hoạt động sinh hoạt hàng ngày. Đây là một trong những bệnh\\nthường gặp ở trẻ em. (1)",\n  "symptoms": "Triệu chứng của cảm lạnh thường bao gồm sổ mũi, nghẹt mũi, hắt hơi, ho, đau\\nhọng, sốt nhẹ, mệt mỏi và đôi khi đau nhức cơ thể nhẹ. Khi nhận thấy con có những\\ntriệu chứng này, ba mẹ hãy đảm bảo cho trẻ nghỉ ngơi nhiều, uống nhiều nước và\\ngiữ vệ sinh cá nhân tốt để ngăn ngừa lây lan virus sang người khác.",\n  "treatment": "Cách điều trị cảm lạnh hướng đến mục tiêu giảm triệu chứng và chăm sóc hỗ trợ để\\nnâng cao phản ứng miễn dịch của cơ thể. Bác sĩ có thể c

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3"
)

In [ ]:
# Optional: standardize dữ liệu cho unsloth
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(
    dataset
)

In [ ]:
dataset

Dataset({
    features: ['messages'],
    num_rows: 11
})

In [ ]:
def formatting_prompts_func(examples):
  convos = examples['messages']
  texts = [
      tokenizer.apply_chat_template(
          convo,
          tokenize = False,
          add_generation_prompt = False).removeprefix('<bos>')
      for convo in convos
  ]
  return {'text': texts}

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['messages', 'text'],
    num_rows: 11
})

In [ ]:
dataset[0]

{'messages': [{'content': 'Mình muốn biết thông tin về bệnh Cảm lạnh thông thường. Hãy trích xuất các phần: tên bệnh, mô tả, triệu chứng và cách điều trị.',
   'role': 'user'},
  {'content': '{\n  "name": "Cảm lạnh thông thường",\n  "definition": "(phổ biến nhất là rhinovirus). Bệnh thường ảnh hưởng đến mũi và cổ họng. Cảm\\nlạnh thông thường không quá nguy hiểm nhưng có thể làm cho trẻ bị khó chịu và\\ngây gián đoạn một số hoạt động sinh hoạt hàng ngày. Đây là một trong những bệnh\\nthường gặp ở trẻ em. (1)",\n  "symptoms": "Triệu chứng của cảm lạnh thường bao gồm sổ mũi, nghẹt mũi, hắt hơi, ho, đau\\nhọng, sốt nhẹ, mệt mỏi và đôi khi đau nhức cơ thể nhẹ. Khi nhận thấy con có những\\ntriệu chứng này, ba mẹ hãy đảm bảo cho trẻ nghỉ ngơi nhiều, uống nhiều nước và\\ngiữ vệ sinh cá nhân tốt để ngăn ngừa lây lan virus sang người khác.",\n  "treatment": "Cách điều trị cảm lạnh hướng đến mục tiêu giảm triệu chứng và chăm sóc hỗ trợ để\\nnâng cao phản ứng miễn dịch của cơ thể. Bác sĩ có thể c

In [ ]:
dataset[0]["text"]

'<start_of_turn>user\nMình muốn biết thông tin về bệnh Cảm lạnh thông thường. Hãy trích xuất các phần: tên bệnh, mô tả, triệu chứng và cách điều trị.<end_of_turn>\n<start_of_turn>model\n{\n  "name": "Cảm lạnh thông thường",\n  "definition": "(phổ biến nhất là rhinovirus). Bệnh thường ảnh hưởng đến mũi và cổ họng. Cảm\\nlạnh thông thường không quá nguy hiểm nhưng có thể làm cho trẻ bị khó chịu và\\ngây gián đoạn một số hoạt động sinh hoạt hàng ngày. Đây là một trong những bệnh\\nthường gặp ở trẻ em. (1)",\n  "symptoms": "Triệu chứng của cảm lạnh thường bao gồm sổ mũi, nghẹt mũi, hắt hơi, ho, đau\\nhọng, sốt nhẹ, mệt mỏi và đôi khi đau nhức cơ thể nhẹ. Khi nhận thấy con có những\\ntriệu chứng này, ba mẹ hãy đảm bảo cho trẻ nghỉ ngơi nhiều, uống nhiều nước và\\ngiữ vệ sinh cá nhân tốt để ngăn ngừa lây lan virus sang người khác.",\n  "treatment": "Cách điều trị cảm lạnh hướng đến mục tiêu giảm triệu chứng và chăm sóc hỗ trợ để\\nnâng cao phản ứng miễn dịch của cơ thể. Bác sĩ có thể chỉ địn

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=6):   0%|          | 0/11 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 11 | Num Epochs = 50 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 16,394,240 of 4,316,473,712 (0.38% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.818600
2,2.202000
3,2.142000
4,1.416500
5,1.960900
6,1.482500
7,1.683800
8,1.404000
9,1.478100
10,1.420100


In [ ]:
# Test prompt
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3"
)
messages = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": "Mình muốn biết thông tin về bệnh Viêm phế quản. tên bệnh, mô tả, triệu chứng và cách điều trị"
    }]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = True,
    return_tensors = "pt",
    return_dict = True,
)
outputs =  model.generate(
    **inputs.to("cuda"),
    max_new_tokens = 300,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nMình muốn biết thông tin về bệnh Viêm phế quản. tên bệnh, mô tả, triệu chứng và cách điều trị<end_of_turn>\n<start_of_turn>model\n{\n  "disease_name": "Viêm phế quản",\n  "definition": "hay còn gọi là nhiễm virus hô hấp hoặc một số nấm/vi khuẩn, viêm phế quản là bệnh khá thường gặp ở trẻ em\\nnhỉn phế quản là bệnh khá thường gặp ở trẻ em.",\n  "symptoms": "Triệu chứng viêm phế quản thường bao gồm ho, khi ho trẻ có thể bị nghẹt thở ho nặng, thở khò khè, số tiền nhẹ, nhức badan không có triệu chứng hoặc có triệu chứng giống với bệnh nhi ở trẻ sơ sinh và các bệnh này rất dễ cản.\\nho",\n  "treatment": "Mục tiêu chính của việc điều trị viêm phế quản là giảm triệu chứng và ngăn ngừa bệnh diễn tiến nghiêm trọng. Bác sĩ có thể kê cho bệnh nhi một số loại thuốc như thuốc giãn phế quản, corticoid để giảm viêm, giảm dịch trong phế quản và ngăn ngừa bệnh trở nên nghiêm trọng hơn."\n}<end_of_turn>']

In [ ]:
b1 luwu toan bo ho so benh an (trieu chung ket luan) bo du lieu trieu chung bo du lieu ket luan
b2: nguoid dung

trien khai: gom benh ly, download nhung benh ly gia lap (kaggle), dung gpt tao ra cac benh ly can ban, len mang download nhung benh ly can BatchedNearestExamplesResults
gia lap benh ly ve nhi, gia lap benh an ve nhi bn tuoi bnh ngya bn khu vuc, mot mo dieu kie, dua vao llm tra ra Las
dua ra mot truong hop cu the
nguoi dung tra loi lai, phan tich
danh gia lai bang exxplanable ai
gioi thieu mot he thong huan luyen sinh vien y, du lieu dung co the sai



In [ ]:
# Test prompt
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3"
)
messages1 = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": "Mình muốn biết thông tin về bệnh Viêm ruột. Các thông tin bao gồm tên bệnh, mô tả, triệu chứng và cách điều trị"
    }]
}]
inputs = tokenizer.apply_chat_template(
    messages1,
    add_generation_prompt = True,
    tokenize = True,
    return_tensors = "pt",
    return_dict = True,
)
outputs =  model.generate(
    **inputs.to("cuda"),
    max_new_tokens = 500,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nMình muốn biết thông tin về bệnh Viêm ruột. Các thông tin bao gồm tên bệnh, mô tả, triệu chứng và cách điều trị<end_of_turn>\n<start_of_turn>model\n{\n  "name": "Viêm ruột",\n  "definition": "Bệnh vi khuẩn\\nrút\\nBé viêm ruột (trẻ < 5 tuổi) do nhiễm các vi khuẩn như Salmonella, Shigella, Escherichia,... Đây là bệnh\\nchuyên khoa trẻ sơ\\nnăng dễ gặp nhất.\\nTriệu chứng\\nCách viêm ruột hình thành là do sự xâm nhập vi khuẩn, ký sinh\\ntrùng hoặc nấm vào thể\\ntrong và làm gây biến chéfung\\nkiểm soát quá trình làm phân\\nchuyển chơntrong các chất độc\\nphản ứng tiêu\\nhóa. Đây là triệu chứng\\nliền với sốt, đau bụng, buồn\\nnào là tiêu\\nrối tiêu hóa. Nếu bé\\ncũng có các triệu chứng sau, ba\\nbé có nguy cơ bị\\nviêm ruột bé cần được chăm\\nsó và tư khắc ngay lập\\nnháng là đau bảptrong cơ bụng, đầy\\nđặc biệt là bé chưa có\\nkiếu thức bình thường, nôn\\nhoái, đi tiêu ra máu hoặc\\nđầy đau (đặc biệt khi bị\\nđầy, bé cũng có thể bị mất\\nkhác cho bạn.\\nCách 

In [ ]:
# Test prompt
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3"
)
messages2 = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": "triệu chứng của bệnh Cúm"
    }]
}]
inputs = tokenizer.apply_chat_template(
    messages2,
    add_generation_prompt = True,
    tokenize = True,
    return_tensors = "pt",
    return_dict = True,
)
outputs =  model.generate(
    **inputs.to("cuda"),
    max_new_tokens = 300,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\ntriệu chứng của bệnh Cúm<end_of_turn>\n<start_of_turn>model\n{\n  "title": "Triệu chứng của bệnh Cúm",\n  "introduction": "Một số triệu chứng của bệnh Cúm có thể cho thấy nguy cơ mắc bệnh cao",\n  "symptoms": "Triệu chứng của bệnh Cúm có thể khác nhau tùy thuộc ở mức độ nghiêm trọng của bệnh\\nbé vẫn có khả năng hoạt động bình thường ngay cả khi có triệu chứng",\n  "symptoms_baby": "Tiêm ngừa. Trẻ được tiêm ngừa cúm từ 6 tháng tuổi và trẻ sơ sinh vẫn có thể được tiêm\\nngập từ 6 tháng bằng loại vắc xin để ngăn ngừa biến chứng của bệnh.",\n  "symptoms_toddler": "Hệ miễn dịch chưa phát triển hoàn thiện.\\n Trẻ em, đặc biệt là trẻ sơ sinh và trẻ nhỏ, có nguy cơ cao mắc biến chứng nghiêm trọng\\n do cúm bao gồm viêm phổi, viêm tai, nhiễm khuẩn\\nxứ dại hoặc có các bệnh lý mắc cơ bản. Ba mẹ lưu ý các triệu chứng của nhiãn.",\n  "symptoms_teen": "Trẻ học sinh và tuổi tiền thiếu nhi có thể tiếp xúc với virus cúm trong môi trường\\nhọc vấn gần gũi với nhiều học sinh

In [ ]:
# Test prompt

messages4 = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": "Mình muốn biết thông tin về bệnh Táo bón. Hãy lấy thông tin về triệu chứng của bệnh Táo bón"
    }]
}]
inputs = tokenizer.apply_chat_template(
    messages4,
    add_generation_prompt = True,
    tokenize = True,
    return_tensors = "pt",
    return_dict = True,
)
outputs =  model.generate(
    **inputs.to("cuda"),
    max_new_tokens = 300,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nMình muốn biết thông tin về bệnh Táo bón. Hãy lấy thông tin về triệu chứng của bệnh Táo bón<end_of_turn>\n<start_of_turn>model\n{\n  "title": "Táo bón là gì? triệu chứng, nguyên nhân và cách phòng ngừa",\n  "introduction": "r\\"n\\nBệnh táo bón thường ảnh hưởng đến trẻ em, thường là do chế độ ăn ít chất xơ hoặc thiếu lượng nước, ít hoạt động thể chất hoặc một số bệnh lý.\\n{\n  \\"definition\\": \\"Táo bón ở trẻ em là tình trạng đi tiêu ít hơn 3 lần một tuần hoặc cứng, khô và khó rít. Trẻ bón có thể bị đau bụng, khó chịu và bị nhiễm trùng do vi khuẩn trong niêm mạc trực tràng do cọ xát do phân cứng và khô.\\",\n  \\"symptoms\\": \\"Triệu chứng táo bón ở trẻ em có thể bao gồm:\\n{\n    \\"irregular\\_bowels\\": \\"đi tiêu không thường xuyên hoặc không thường xuyên (ít hơn 3 lần một tuần, đặc biệt ở trẻ em)\\",\n    \\"hard\\_stools\\": \\"bỉm cứng, khô hoặc bé rít khi phân khi đi đại tiện\\",\n    \\"stomach\\_cramps\\": \\"trẻ bị đau bụng hoặc khó chịu ở bụn

In [ ]:
# Test prompt

messages5 = [{
    "role": "user",
    "content": [{
        "type": "text",
        "text": "Mình muốn biết thông tin về bệnh Viêm xoang. Hãy lấy thông tin cách điều trị của bệnh Viêm xoang. Trả lời đúng theo trong pdf đã học, không thêm, không bớt, không bịa nội dung, không trả lời những cái không cần thiết"
    }]
}]
inputs = tokenizer.apply_chat_template(
    messages5,
    add_generation_prompt = True,
    tokenize = True,
    return_tensors = "pt",
    return_dict = True,
)
outputs =  model.generate(
    **inputs.to("cuda"),
    max_new_tokens = 300,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nMình muốn biết thông tin về bệnh Viêm xoang. Hãy lấy thông tin cách điều trị của bệnh Viêm xoang. Trả lời đúng theo trong pdf đã học, không thêm, không bớt, không bịa nội dung, không trả lời những cái không cần thiết<end_of_turn>\n<start_of_turn>model\n{\n"type": "page",\n"metadata": {\n"pageId": "page-24",\n"pageNumber": 24\n},\n"content": "Viêm xoang là một trong những bệnh thường gặp ở cả trẻ em và người trưởng thành. Đây là bệnh thường gặp ở trẻ em và các bé trưởng thành đặc biệt là mùa thu và đông đến. Bẹnn xoang là một trong những bệnh thường gặp ở cả trẻ em và người trưởng thành. Đây là bệnh thường gặp ở trẻ em và các bé trưởng thành đặc biệt là mùa thu và đông đến. Viêm xoang là một trong những bệnh thường gặp ở cả trẻ em và người trưởng thành. Đây là bệnh thường gặp ở trẻ em và các bé trưởng thành đặc biệt là mùa thu và đông đến.\\n\\n**1. Triệu chứng của bệnh viêm xoang?**\\n\\nKhi có triệu chứng viêm xoang, trẻ có thể bị\\n*   Đổ mồ hôi như sau là